# Exploring Physical Properties of an Electrolye

This notebook is intended to serve as a tutorial of how to setup and execute an end-to-end a bespoke HTMD workflow using atomate2-openmm. The end-to-end example will be in terms of exploring the physical properties of a constrained randok sample of electrolyte formulations as part of a developing lithium ion batteries.

Let's first frame the problem. Suppose we are looking to develop an electrolyte formulaton for a lithium ion battery. Basic domain knowledge on electrolyte formulating suggest that a battery electrolyte can typically consist of components tabulated below with concentrations given in moles.

| Component                            | Concentration (number of molecules) | SMILES String          |
|--------------------------------------|-------------------------------------|------------------------|
| Lithium Hexafluorophosphate (LiPF6)  | 500 - 1000                          | [Li+].[F-]P(F)(F)(F)(F)F |
| Ethylene Carbonate (EC)              | 5000 - 10000                        | CC(=O)OCC               |
| Dimethyl Carbonate (DMC)             | 5000 - 10000                        | CC(=O)OC(=O)C            |
| Diethyl Carbonate (DEC)              | 2000 - 5000                         | CCOC(=O)CC               |
| Vinylene Carbonate (VC)              | 100 - 1000                          | C1C(=O)OC=C1             |
| Fluoroethylene Carbonate (FEC)       | 100 - 1000                          | C1C(=O)OCCF1             |


What we would like to do is a constrained uniform random sample of 1000 random formulations electrolyte recipes and run them through an HTMD workflow using atomate2-openmm and analyze the results (for property X). To begin this process we first need to convert the collection of simulation we would like to run into the XML files that are used as simulation input. Creating XML files is cumbersome, so we can utilize pymatgen-io-openmm to generate these XML files from basic information like chemical structure and the number of molcules to include.

In [2]:
from pymatgen.io.openmm.generators import OpenMMSolutionGen 

In [4]:
openmm_sol_gen = OpenMMSolutionGen()

In [8]:
openmm_set = openmm_sol_gen.get_input_set(
    input_mol_dicts=[
        {"smile": "[Li+].[F-]P(F)(F)(F)(F)F", "count": 500},
        {"smile": "CC(=O)OCC", "count": 5000},
        {"smile": "CC(=O)OC(=O)C", "count": 5000},
        {"smile": "CCOC(=O)CC", "count": 200},
        {"smile": "C1C(=O)OC=C1", "count": 100},
        {"smile": "C1C(=O)OCCF1", "count": 100},
    ],
    density=1.0,
)

[23:27:49] Explicit valence for atom # 1 F, 2, is greater than permitted


KeyError: 'smile'